# Generate Predictions from Trained Model

In [ ]:
import os
from os import listdir, walk
from os.path import join
from keras_retinanet.bin.train import create_models
from keras_retinanet.models import backbone,convert_model
from keras_retinanet.utils.config import read_config_file,parse_anchor_parameters
import numpy as np
import keras
import math
import tensorflow as tf
tf.set_random_seed(31)
np.random.seed(17)

In [ ]:
b = backbone('resnet50')

In [ ]:
weights = b.download_imagenet()
model, training_model, prediction_model = create_models(
            backbone_retinanet=b.retinanet,
            num_classes=2,
            weights=weights,
            multi_gpu=False,
            freeze_backbone=False,
            lr=1e-3,
            config=read_config_file('config.ini')
        )

# Load and Convert Model

Training model needs to be converted to inference model

In [ ]:
training_model.load_weights('saved/resnet50_csv_70.h5')

In [ ]:
infer_model = convert_model(training_model,anchor_params=parse_anchor_parameters(read_config_file('config.ini')))

# Create Generator to load images

In [ ]:
from keras_retinanet.utils.image import read_image_bgr, preprocess_image, resize_image
from tqdm import tqdm

def test_gen(image_ids, bs = 32, size=672,test = True):
    imgs = []
    scale = None
    idx = 0
    if test:
        path = 'test_data_images/images/'
    else:
        path = 'training_data/images/'
    
    while idx < len(image_ids):
        if len(imgs) < bs:
            imgs.append(resize_image(preprocess_image(read_image_bgr(path + image_ids[idx] + '.jpg')),min_side=size,max_side=size)[0])            
            if scale is None:
                scale = resize_image(preprocess_image(read_image_bgr(path + image_ids[idx] + '.jpg')),min_side=size,max_side=size)[1]
            idx += 1
        else:
            yield np.array(imgs),scale
            imgs = []
            
            
    if len(imgs) > 0:
        yield np.array(imgs),scale        

# Save Predictions

In [ ]:
_,_,image_ids = next(walk('test_data_images/images'))
image_ids = [i[:-4] for i in image_ids]
image_ids = sorted(image_ids)

os.system('mkdir predictions')

iter_num = 0
test_bs = 8


for imgs,scale in tqdm(test_gen(image_ids,bs=test_bs),total=math.ceil(len(image_ids)/test_bs)):
    boxes, scores, labels = infer_model.predict_on_batch(imgs)
    boxes /= scale
    for img_num in range(len(imgs)):
        with open('predictions/' + image_ids[(iter_num*test_bs) + img_num] + '.txt', 'w') as f:
            for box, score, label in zip(boxes[img_num], scores[img_num], labels[img_num]):
                # scores are sorted so we can break
                if score < 0:
                    break
                f.write(f'{label + 1} {score} {int((box[1]))} {int((box[0]))} {int((box[3]))} {int((box[2]))} \n')
    iter_num += 1

In [ ]:
os.system('zip -r predictions.zip predictions/')